# EX 4 - 
In this problem you are asked to implement a program that builds decision tree
from categorical attributes and two-class classication tasks. The programming
part requires building a tree from a training dataset and classifying unseen
instances.

In [5]:
!pip install graphviz

In [6]:
import numpy as np
import pandas as pd
import glob
import graphviz
import os


In [7]:
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
eps = np.finfo(float).eps
dot=graphviz.Digraph(format='png')
id = 0


In [8]:
def find_entropy(df,feature=None):
    target = df.keys()[-1] 
    if (feature is None):
        entropy = 0
        values = df[target].unique()
        for value in values:
            fraction = df[target].value_counts()[value] / len(df[target])
            entropy += -fraction * np.log2(fraction)
        return entropy
    else:
        target_variables = df[target].unique()  
        variables = df[feature].unique() 
        tot_entropy = 0
        for variable in variables:
            entropy = 0
            for target_variable in target_variables:
                num = len(df[feature][df[feature] == variable][df[target] == target_variable])
                denominator = len(df[feature][df[feature] == variable])
                p = num / (denominator + eps)
                entropy += -p * np.log2(fraction + eps)
            p2 = denominator / len(df)
            tot_entropy += -p2 * entropy
        return abs(tot_entropy)


In [9]:
def find_best_feature(df):
    info_gain = []
    ent=find_entropy(df)
    for key in df.keys()[:-1]:
        info_gain.append(ent - find_entropy(df, key))
    return df.keys()[:-1][np.argmax(info_gain)],ent,info_gain[np.argmax(info_gain)]


def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)

def decision_tree_build (df, possible_values, tree=None):
    target = df.keys()[-1]  

def decision_tree_build (df, possible_values, tree=None):
    target = df.keys()[-1]  

    if len(df.columns) == 1:
        return df[target].value_counts().idxmax()
    
    node,ent,ig = find_best_feature(df)

    if tree is None:
        tree = {}
        # 
        tree[node] = {}
    for value in possible_values[node]:

        subtable = get_subtable(df, node, value).drop(columns=node)
        clValue, counts = np.unique(subtable[target], return_counts=True)

        if len(counts) == 1:  
            tree[node][value] = clValue[0]
        else:
            if subtable.empty:
                tree[node][value] = df[target].value_counts().idxmax()
            else:
                tree[node][value] = decision_tree_build(subtable, possible_values) 
    tree["details"]={"entropy": ent,"information_gain": ig}

    return tree


In [10]:
def to_list(file):
    temp = []
    possible_values = {}
    with open(file, 'r') as f:
        for line in f:
            if not line.startswith(('//', '%%', '##')):
                temp.append(line)
            if line.startswith('##'):
                line = line.rstrip()
                line = line.split(',')
                possible_values[line[1]] = line[2:]
    return possible_values, temp


In [15]:

def loadData():
    train, val, test = [], [], []
    for file in glob.glob("*.txt"):
        if ("test" in file):
            _, test = to_list(file)
        if ("train" in file):
            possible_values, train = to_list(file)
        if ("val" in file):
            _, val = to_list(file)

    return possible_values, train, val, test


def toDF(data):
    rows = []
    for line in data:
        row = []
        a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, y = str(line).split(',')
        row.append(a1)
        row.append(a2)
        row.append(a3)
        row.append(a4)
        row.append(a5)
        row.append(a6)
        row.append(a7)
        row.append(a8)
        row.append(a9)
        row.append(a10)
        row.append(y[0])
        rows.append(row)

    col_name = ["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10", "Y"]
    df = pd.DataFrame(rows, columns=col_name)
    return df


In [16]:
def predict(sample, tree):
    for nodes in tree.keys():
        if(nodes != "details"):
    
            value = sample[nodes]
            tree = tree[nodes][value]
    
            prediction = 0
    
            if type(tree) is dict:
                prediction = predict(sample, tree)
            else:
                prediction = tree
                break

    return prediction


def print_accuracy(val_df, tree):
    accuracy = 0
    for i, val in val_df.iterrows():
        result = predict(val, tree)
        if result == val[-1]:
            accuracy += 1
    return accuracy / len(val_df)



def display_graph(tree, parent=None, label=None):
    global id
    id += 1
    parent_id = id
    feature = list(tree.keys())[0]
    entropy = f'{tree.get("details").get("entropy"):.5f}'
    ig = f'{tree.get("details").get("information_gain"):.5f}'
    information="{}\nEntropy: {}\nInformation gain: {}".format(feature,entropy,ig)
    
    dot.node(str(parent_id), label=information,shape='box')

    if parent is not None and label is not None:
        dot.edge(str(parent), str(parent_id), label=label)

    for v in list(tree[feature].keys()):
        if type(tree[feature][v]) is dict:
            display_graph(tree[feature][v], parent=parent_id, label=v)
        else:
            id += 1
            dot.node(str(id), label=tree[feature][v])
            dot.edge(str(parent_id), str(id), label=v)
            

In [18]:
def main():
    
    possible_values, train, val, test = loadData()
    train_df = toDF(train)
    val_df = toDF(val)
    
    tree = decision_tree_build(train_df, possible_values)
    print(print_accuracy(val_df,tree))
    
    
    display_graph(tree)
    dot.render('plot', view=True)  
    
if(__name__=="__main__"):
    main()